In [1]:
import pandas as pd
import datetime
import MySQLdb
import os
import warnings
warnings.filterwarnings("ignore")

In [32]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [35]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [11]:
db.close()

In [31]:
dbname = 'ta_oi_data'
cursor = db.cursor()
sql = "ALTER TABLE " + dbname + ".v_rank ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_rank ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_rank ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_increment ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_increment ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_increment ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".v_volume ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".b_oi ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)
sql = "ALTER TABLE " + dbname + ".s_oi ADD COLUMN `前海期货` TEXT"
cursor.execute(sql)

8843

In [33]:
dbname = 'ta_oi_data'
cursor = db.cursor()

sql = "delete from " + dbname + ".total_volume where Date >= '2018-3-5' "
cursor.execute(sql)


sql = "delete from " + dbname + ".v_rank where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".b_rank where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".s_rank where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".v_volume where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".b_oi where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".s_oi where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".v_increment where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".b_increment where Date >= '2018-3-5' "
cursor.execute(sql)
sql = "delete from " + dbname + ".s_increment where Date >= '2018-3-5' "
cursor.execute(sql)

13

In [37]:
# contract_list = ['cf','cy','er','fg','jr','lr','ma','oi','pm','ri','rm','rs','sf','sm','sr','ta','wh','tc'] #AP
contract_list = ['tc']
FODER_DIR = "../各交易所期貨持倉量/鄭商所/"

for contract_id in contract_list:
    
    print(contract_id)
    cursor = db.cursor()
    
    DATA_DIR = FODER_DIR + contract_id
    
    dbname1 = contract_id + "_oi_data"
    dbname2 = contract_id + "_oi_original_data"
    
    company_order = pd.read_excel("../期貨公司名稱/鄭商所/" + contract_id + "_company_name.xlsx",header=None)
    
#     sql = "CREATE DATABASE IF NOT EXISTS " + dbname1
#     cursor.execute(sql)
#     sql = "CREATE DATABASE IF NOT EXISTS " + dbname2
#     cursor.execute(sql)
    
#     create_total_volume(dbname1)
    
#     V_info_create(dbname1,company_order)
#     B_info_create(dbname1,company_order)
#     S_info_create(dbname1,company_order)
    
    for filename in os.listdir(DATA_DIR):
        
        split_name = filename.split("_")
        getname = split_name[0]
        date = split_name[1].split(".csv")[0]
        
        date = datetime.datetime.strptime(date,'%Y%m%d')
    
        if (date > datetime.datetime(2018,3,5)):
            
            df = pd.read_csv(DATA_DIR+"/"+filename,engine="python",encoding="utf-8-sig")
            df = df.where((pd.notnull(df)), None)
            date = date.strftime('%Y-%m-%d')
            
            for i in range(len(df)):
                for j in range(10):
                    if(str(df.loc[i][j]).strip() == "-"):
                        df.set_value(i, j , None, takeable=True)  #塞原始資料

                    elif((j==2 or j==3 or j==5 or j==6 or j==8 or j==9) and df.loc[i][j] != None):
                        df.set_value(i, j, str(df.iloc[i][j]).replace(',',''),takeable=True)


#             total_insert(df, filename.split(".csv")[0], dbname2, 0)
            total_volume_insert(df, date, getname, dbname1)
        
            V_info_insert(df, getname, date, dbname1)
            B_info_insert(df, getname, date, dbname1)
            S_info_insert(df, getname, date, dbname1)

tc


In [28]:
b = []
c = []

a = df['1']
a = pd.DataFrame(list(a))

for i in range(len(a) - 1):
    c.append(a.loc[i][0])

for i in range(len(company_order)):
    b.append(company_order.loc[i][0])
    
for x in c:
    if x.strip() not in b:
        print(x)

前海期货    


In [3]:
def total_insert(data,tbname,databaseName,start_index):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + "." + tbname + "(\
    Rank INT(5),\
    Member CHAR(10),\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Member1 CHAR(10),\
    Buy_Volume INT(10),\
    Increment_1 FLOAT(5),\
    Member2 CHAR(10),\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5) )"

    cursor.execute(sql)
    value=[]

    for i in range(start_index,len(data)-1):      
        value.append((str(data.loc[i][0]).strip(),str(data.loc[i][1]).strip(),str(data.loc[i][2]).strip(),str(data.loc[i][3]).strip(),\
                      str(data.loc[i][4]).strip(),str(data.loc[i][5]).strip(),str(data.loc[i][6]).strip(),str(data.loc[i][7]).strip(),\
                      str(data.loc[i][8]).strip(),str(data.loc[i][9]).strip()))
    
    sql = "INSERT INTO " + databaseName + "." + tbname +" VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

In [4]:
def create_total_volume(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume (Date DATE ,Contract_id TEXT(5), \
    V_Valume FLOAT,V_Increment FLOAT,\
    B_Valume FLOAT,B_Increment FLOAT,\
    S_Valume FLOAT,S_Increment FLOAT)"

    cursor.execute(sql)
    
def total_volume_insert(data, table_date, contract_id, databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
    sql = "INSERT INTO " + databaseName + ".total_volume VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
    
    if(str(df.loc[total_len][1]) == 'None' or str(df.loc[total_len][1]).strip() == ""):          
        cursor.execute(sql,(table_date,contract_id,str(data.loc[total_len][2]).strip(),str(data.loc[total_len][3]).strip(),\
                            str(data.loc[total_len][5]).strip(),str(data.loc[total_len][6]).strip(),\
                            str(data.loc[total_len][8]).strip(),str(data.loc[total_len][9]).strip()))
    else:
        cursor.execute(sql,(table_date,contract_id,str(data.loc[total_len][1]).strip(),str(data.loc[total_len][2]).strip(),\
                            str(data.loc[total_len][4]).strip(),str(data.loc[total_len][5]).strip(),\
                            str(data.loc[total_len][7]).strip(),str(data.loc[total_len][8]).strip()))
        db.commit()
        cursor.close()

In [5]:
def V_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE, Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def V_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"


    sql = "INSERT INTO " + databaseName + ".v_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(0,len(data)-1):
        if(data.loc[i][1] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]) + "=" + str(data.loc[i][0]) + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(0,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]) + "=" + str(data.loc[i][2]) + \
                  " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(0,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + str(data.loc[i][1]) + "=" + str(data.loc[i][3]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

def B_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def B_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()
 
    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(0,len(data)-1):
        if(data.loc[i][4] == None):
            continue  
        else:
            sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][4]) + "=" + str(data.loc[i][0]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(0,len(data)-1):
        if(data.loc[i][4] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][4]) + "=" + str(data.loc[i][5]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(0,len(data)-1):
        if(data.loc[i][4] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".b_increment SET " + str(data.loc[i][4]) + "=" + str(data.loc[i][6]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()

## S名次、賣單量、增減

def S_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE , Contract_id VARCHAR(50), " + name + ",\
           PRIMARY KEY(Date, Contract_id)) ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

def S_info_insert(data, contract_name, table_date, databaseName):
    cursor = db.cursor()

    empty = ""
    empty = empty + "'" + table_date + "', '" + contract_name + "'"
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date, Contract_id) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(0,len(data)-1):
        if(data.loc[i][7] == None):
            continue  
        
        else:
            sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][7]) + "=" + str(data.loc[i][0]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()    

    for i in range(0,len(data)-1):
        if(data.loc[i][7] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][7]) + "=" + str(data.loc[i][8]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    for i in range(0,len(data)-1):
        if(data.loc[i][7] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName +".s_increment SET " + str(data.loc[i][7]) + "=" + str(data.loc[i][9]) + \
                 " WHERE Date='" + table_date + "' and Contract_id = '" + contract_name + "'"
            cursor.execute(sql)
            db.commit()

    cursor.close()

In [42]:
db = MySQLdb.connect("localhost","root","123456","wh_oi_original_data")
db.set_character_set('gbk')

In [43]:
cursor = db.cursor()
sql = 'show tables'
cursor.execute(sql)
data = cursor.fetchall()

In [44]:
company_list = []
for table in data:
    sql = 'select Member,Member1,Member2 from ' + table[0]
    cursor.execute(sql)
    name = cursor.fetchall()   
    for i in name:
        for j in range(3):
            if (i[j] not in company_list and i[j]!=None):
                company_list.append(i[j])

KeyboardInterrupt: 

In [32]:
a = pd.DataFrame(company_list)
a.to_excel("../期貨公司名稱/wh_company_name.xlsx",index=None,header=None)

In [7]:
df = pd.read_excel("../期貨公司名稱/鄭商所/123.xlsx",sheet_name=1)

In [8]:
df

,878
0,8787
1,8754
2,878
